In [8]:
import pandas as pd

In [9]:
df = pd.read_excel("./WineKMC.xlsx", sheet_name = 0)

df.columns = ['offer_id', 'campaign','varietal','min_qty','discount','origin','past_peak']
df.head(5)

c:\Users\zaraa\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,offer_id,campaign,varietal,min_qty,discount,origin,past_peak
0,1,January,Malbec,72,56,France,False
1,2,January,Pinot Noir,72,17,France,False
2,3,February,Espumante,144,32,Oregon,True
3,4,February,Champagne,72,48,France,True
4,5,February,Cabernet Sauvignon,144,44,New Zealand,True


In [10]:
df_transac = pd.read_excel("./WineKMC.xlsx", sheet_name = 1)

df_transac.columns = ['customer_name','offer_id']

df_transac['n'] = 1

df_transac.head(5)

c:\Users\zaraa\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,customer_name,offer_id,n
0,Smith,2,1
1,Smith,24,1
2,Johnson,17,1
3,Johnson,24,1
4,Johnson,26,1


In [11]:
df_merged = pd.merge(df, df_transac)

matrix = df_merged.pivot_table(index=['customer_name'], columns=['offer_id'], values = ['n'])

matrix = matrix.fillna(0).reset_index()

x_cols = matrix.columns[1:]

In [12]:
from sklearn.cluster import KMeans

cluster = KMeans(n_clusters=5)

#slice matrix to only include 0/1 indicator columns
matrix['cluster'] = cluster.fit_predict(matrix[matrix.columns[2:]])

matrix.cluster.value_counts()

c:\Users\zaraa\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


cluster
2    31
4    19
3    19
0    16
1    15
Name: count, dtype: int64

In [13]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
matrix['x'] = pca.fit_transform(matrix[x_cols])[:,0]
matrix['y'] = pca.fit_transform(matrix[x_cols])[:,1]
matrix = matrix.reset_index()

customer_clusters = matrix[['customer_name','cluster','x','y']]
customer_clusters.head()

,customer_name,cluster,x,y
offer_id,,,,
0,Adams,4,1.007580,0.108215
1,Allen,2,-0.287539,0.044715
2,Anderson,0,-0.392032,1.038391
3,Bailey,1,0.699477,-0.022542
4,Baker,3,0.088183,-0.471695
